## Do not execute this notebook on its own

This notebook is to be called by other notebooks in the subfolders 

In [ ]:
# plot_for_monte.ipynb


In [ ]:
# lim_dist_ave = None
# lim_dis_std = None
# lim_dist_point = None
# lim_dist_q = None
# lim_dist_g = None
# lim_std_T = None
# print_integ = True

# if 'lim_std_T' 'lim_std_Q' 'lim_std_G' not in locals():
lim_std_Q = locals().get('lim_std_Q', None)
lim_std_G = locals().get('lim_std_G', None)
lim_std_T = locals().get('lim_std_T', None)

# if scaling does not exist create scaling = [1.,1.]
if 'scaling' not in locals():
    scaling = [1.,1.]

# if 'color_lims' not in locals():
#     color_lims = None

# a = !ls {name_vtk_integ}*
# a = a[0][0]
# if a == 'l':
#     print_integ = False

In [ ]:
# if color_lims:
#     lim_std_T = [0., 0.25]
    

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()


name_vtk_monte_errors = "out_monte_errors_"+csv_name[:-4]

name_vtk_monte_integ = "out_monte_integ_"+csv_name[:-4]

name_vtk_monte_sigma = "out_monte_sigma_"+csv_name[:-4]

name_vtk_monte_integ_mean_var = "out_integ_pts_mean_var_"+csv_name[:-4]

In [ ]:
print("plotting results for Temperature standard deviation")
params.show_file = name_vtk_monte_sigma
params.show_field = "SIGMA_T"
params.show_field_scale = 1./scaling[1]
params.p_cmap = color_std_temperature
params.clim = lim_std_T
params.show_edges = False
params.p_save = prefix+"T_std.pdf"
show_results(params)

In [ ]:
print("plotting results for Flux standard deviation x")
params.show_file = name_vtk_monte_sigma
params.show_field = "SIGMA_FLUX_X"
params.show_field_scale = 1./scaling[0]
params.p_cmap = color_std_flux
params.clim = lim_std_Q
params.show_edges = False
params.p_save = prefix+"Q_std_x.pdf"
show_results(params)

In [ ]:
print("plotting results for Flux standard deviation y")
params.show_file = name_vtk_monte_sigma
params.show_field = "SIGMA_FLUX_Y"
params.show_field_scale = 1./scaling[0]
params.p_cmap = color_std_flux
params.clim = lim_std_Q
params.show_edges = False
params.p_save = prefix+"Q_std_y.pdf"
show_results(params)


params.show_field_scale = 1.

In [ ]:
# for SIGMA_GRAD_X and SIGMA_GRAD_Y

print("plotting results for Gradient standard deviation x")
params.show_file = name_vtk_monte_sigma
params.show_field = "SIGMA_GRAD_X"
params.p_cmap = color_std_gradient
params.clim = lim_std_G
params.show_edges = False
params.p_save = prefix+"G_std_x.pdf"
show_results(params)


In [ ]:
print("plotting results for Gradient standard deviation y")
params.show_file = name_vtk_monte_sigma
params.show_field = "SIGMA_GRAD_Y"
params.p_cmap = color_std_gradient
params.clim = lim_std_G
params.show_edges = False
params.p_save = prefix+"G_std_y.pdf"
show_results(params)

## new plotting

In [ ]:
def show_results_magnitude(params):
    out_to_vtk = !ls -c1 {params.show_file}*vtk
    last_file=out_to_vtk[0]

    p = pv.Plotter(notebook=True)

    mesh = pv.read(last_file[:-3] + "vtk")
    warped_mesh = mesh

    if params.warp_field_scalar:
        # p.add_mesh(mesh, show_scalar_bar = False, opacity=0.2, color='gray', smooth_shading=True,)
        warped_mesh = mesh.warp_by_scalar(scalars=params.warp_field_scalar, factor=params.warp_factor)
    
    if params.warp_field_vector:
        # p.add_mesh(mesh, show_scalar_bar = False, opacity=0.2, color='gray', smooth_shading=True,)
        warped_mesh = mesh.warp_by_vector(vectors=params.warp_field_vector, factor=params.warp_factor)

    if params.show_edges:
        warped_mesh=warped_mesh.shrink(0.95)
    
    jupyter_backend='ipygany'

    # Check if the field is in point data or cell data
    if params.show_field in warped_mesh.point_data:
        data_location = warped_mesh.point_data
    elif params.show_field in warped_mesh.cell_data:
        data_location = warped_mesh.cell_data
    else:
        raise KeyError(f"Field '{params.show_field}' not found in point or cell data.")

    # Check if params.show_field_2 is defined and calculate sqrt(f1^2 +f2^2) if so
    if (params.show_field_2):
        if (params.field_part < 0):
            scalars1 = data_location[params.show_field]
            scalars2 = data_location[params.show_field_2]
        
        scalars = np.sqrt(scalars1**2 + scalars2**2)
        if params.show_field_diff_abs:
            scalars = np.abs(scalars)

    else:
        scalars = data_location[params.show_field]

    scalars = scalars * params.show_field_scale 

    if params.show_ori_shape:
        p.add_mesh(mesh, component=None, smooth_shading=True, opacity=0.5, color='gray')


    # scalar_bar_title = params.show_field_name if params.show_field_name else params.show_field
    # # Define the arguments for the scalar bar
    # scalar_bar_args = {
    #     # 'n_labels': 8,
    #     'position_x': 0.2,  
    #     'title': scalar_bar_title,
    # }

    scalar_bar_args = p_settings(p, params)

    p.add_mesh(warped_mesh, scalars=scalars, component=None, smooth_shading=False, cmap=params.p_cmap, clim = params.clim, show_scalar_bar=True, opacity=0.9, scalar_bar_args=scalar_bar_args)

    # if params.show_scalar_bar:
    #     scalar_bar_title = params.show_field if params.show_field else "Scalar Field"
    #     p.add_scalar_bar(title=scalar_bar_title, color='black')
 
    p.camera_position = params.camera_position
    
    p.enable_parallel_projection()
    p.enable_image_style()
    
    # p.show(jupyter_backend=jupyter_backend)
    # if params.p_save:
    #     p.save_graphic(params.p_save) 

    p_save_crop(p,params)

In [ ]:
# plot results for the magnitude of the gradient standard deviation
print("plotting results for Gradient standard deviation magnitude")
params.show_file = name_vtk_monte_sigma
params.show_field_name = "SIGMA_GRAD_MAG"
params.show_field = "SIGMA_GRAD_X"
params.show_field_2 = "SIGMA_GRAD_Y"
params.p_cmap = color_std_gradient
params.clim = lim_std_G
params.show_edges = False
params.p_save = prefix+"G_std_mag.pdf"
show_results_magnitude(params)


In [ ]:
# plot results for the magnitude of the flux standard deviation
print("plotting results for Flux standard deviation magnitude")
params.show_file = name_vtk_monte_sigma
params.show_field_name = "SIGMA_FLUX_MAG"
params.show_field_scale = 1./scaling[0]
params.show_field = "SIGMA_FLUX_X"
params.show_field_2 = "SIGMA_FLUX_Y"
params.p_cmap = color_std_flux
params.clim = lim_std_Q
params.show_edges = False
params.p_save = prefix+"Q_std_mag.pdf"
show_results_magnitude(params)


In [ ]:
# reset params values
params.show_field_name = None
params.show_field_2 = None
params.clim = None

params.show_field_scale = 1.


## old

In [ ]:
# print("plotting results for grad(T)")
# params.show_file = name_vtk_result
# params.show_field = "T"
# params.field_part = 10
# params.warp_field_scalar = ""
# params.p_cmap = color_gradient
# params.show_edges = True
# params.p_save = prefix+"gradT.pdf"
# show_results(params)
# params.field_part = -1

In [ ]:
# print("plotting results for Gradient")
# params.show_file = name_vtk_result
# params.show_field = "G"
# params.warp_field_scalar = ""
# params.p_cmap = color_gradient
# params.show_edges = True
# params.p_save = prefix+"G.pdf"
# show_results(params)

In [ ]:
# print("plotting results for Flux (result scale)")
# params.show_file = name_vtk_result
# params.show_field = "Q"
# params.warp_field_scalar = ""
# params.p_cmap = color_flux
# params.show_edges = True
# params.p_save = prefix+"Q.pdf"
# show_results(params)

# print("plotting results for Flux (dataset scale)")
# params.show_file = name_vtk_result
# params.show_field = "Q"
# params.show_field_scale = scaling[0]
# params.warp_field_scalar = ""
# params.p_cmap = color_flux
# params.show_edges = True
# params.p_save = prefix+"Q_scaled.pdf"
# show_results(params)
# params.show_field_scale = 1.0

In [ ]:
# print("DD_DISTANCE_AVE")
# params.show_file = name_vtk_error
# params.show_field = "DD_DISTANCE_AVE"
# params.show_edges = True
# params.p_cmap = 'Purples'
# params.clim = lim_dist_ave
# params.p_save = prefix+"dis_ave.pdf"
# show_results(params)

# print("DD_DISTANCE_VAR")
# params.show_field = "DD_DISTANCE_VAR"
# params.p_cmap = "Blues"
# params.clim = lim_dis_std
# params.p_save = prefix+"dis_var.pdf"
# show_results(params)
# params.p_save = None

In [ ]:
# if print_integ:
#     print("plotting results for Point_distance")
#     params.show_file = name_vtk_integ
#     params.show_field = "Point_distance"
#     params.clim = lim_dist_point
#     # params.clim = None
#     # TODO: check if this is correct
#     params.p_save = prefix+"dis_point.pdf"
#     params.p_cmap = 'Purples'
#     show_resulting_points(params)

In [ ]:
# if print_integ:
#     print("Q - Q_STAR (dataset scale)")
#     params.show_field_scale = 1.0
#     params.show_field_scale_2 = 1.0
#     params.show_file = name_vtk_integ
#     params.field_part = 1
#     params.show_field = "Q"
#     params.show_field_2 = "Q_STAR"
#     params.clim = lim_dist_q
#     params.p_cmap = 'PuBu'
#     params.p_save = prefix+"dis_Q_Qstar_y.pdf"
#     show_resulting_points(params)

#     print("Q - Q_STAR (result scale)")
#     params.show_field_scale = 1./scaling[0]
#     params.show_field_scale_2 = 1./scaling[0]
#     if color_lims:
#         params.clim = [lim * (1.0/scaling[0]) for lim in lim_dist_q]
#     params.p_save = prefix+"dis_Q_Qstar_y_scaled.pdf"
#     show_resulting_points(params)
#     params.show_field_2 = None
#     params.field_part = -1
#     params.p_save = None
#     params.show_field_scale = 1.0
#     params.show_field_scale_2 = 1.0

In [ ]:
# if print_integ:
#     print("GRAD(P) - GRAD(P)_STAR")
#     params.show_file = name_vtk_integ
#     params.show_field = "GRAD(P)"
#     params.show_field_2 = "GRAD(P)_STAR"
#     params.clim = lim_dist_g
#     params.p_cmap = "YlGnBu"
#     params.field_part = 1
#     params.p_save = prefix+"dis_G_Gstar_y.pdf"
#     show_resulting_points(params)
#     params.show_field_2 = None
#     params.p_save = None
#     params.field_part = -1

In [ ]:
# if print_integ:
#     print("T - P_STAR (dataset scale)")
#     params.show_file = name_vtk_integ
#     params.show_field = "T"
#     params.clim = lim_dist_T
#     # params.show_field_scale = scaling[1]
#     params.show_field_2 = "P_STAR"
#     params.p_cmap = "Reds"
#     params.p_save = prefix+"dis_T_Pstar.pdf"
#     show_resulting_points(params)

#     print("T - P_STAR (result scale)")
#     params.show_field_scale = 1.0
#     params.show_field = "T"
#     params.show_field_2 = "P_STAR"
#     if color_lims:
#         params.clim = lim_dist_T/scaling[1]
#     params.show_field_scale = 1./scaling[1]
#     params.show_field_scale_2 = 1./scaling[1]
#     params.p_save = prefix+"dis_T_Pstar_full_scale.pdf"
#     show_resulting_points(params)
#     params.show_field_scale = 1.0
#     params.show_field_scale_2 = 1.0
#     params.show_field_2 = None
#     params.p_save = None
    

In [ ]:

params.show_field_scale = 1.0
params.show_field_scale_2 = 1.0
params.clim = None


# stop the display
display.stop()
